# Descenso al fondo de un cráter en Marte 

- Diseño de agentes inteligentes
- TC2032.101
- Profesor: Juan Emanuel Ledesma Martínez
- Equipo 7:

| Alumno | Matrícula |
| ---- | ---- |
| Juan Pablo Echeagaray González | A00830646 |


Fecha: 13 de marzo del 2022

## Entregable

Se teoriza que en el fondo de algunos de los cráteres de la superficie marciana es posible encontrar agua y otros elementos interesantes que podrían responder a la pregunta de si en Marte alguna vez hubo vida o si la hay en estos momentos. Por ello, es interesante ver la posibilidad de que alguno de los Rover exploradores logre navegar de manera segura hasta estos cráteres y llegue al fondo de los mismos.

Considere que hay un grupo de ingenieros que desean determinar el tipo de explorador que podría llevar a cabo la tarea de descender por un cráter de manera exitosa. Para ello, quieren realizar una serie de simulaciones utilizando como ejemplo el siguiente cráter de la superficie marciana extraído del sitio de [HiRISE](https://www.uahirise.org/dtm/).

![](img/2022-03-14-15-02-24.png)

![](img/2022-03-14-15-02-38.png)

![](img/2022-03-14-15-02-51.png)

La información mostrada con anterioridad se puede representar con una imagen de falso color como la que se muestra a continuación. El color de cada pixel de la imagen representa la altura en dicha posición.

![](img/2022-03-14-15-03-12.png)

Los datos de este cráter tienen dimensiones de 7163 pixeles x 10770 pixeles. Cada pixel representa un área de 1.0045 metros x 1.0045 metros. Los datos mostrados representan la profundidad medida con instrumentos satelitales para cada pixel. El nivel más bajo corresponde a los 0 metros, y se representa con rojo. Los datos mostrados se pueden descargar aquí [Descargar aquí](https://experiencia21.tec.mx/courses/241470/files/78461918?wrap=1) . Para escalar los datos y generar una matriz de numpy utilicen este script [Descargar script](https://experiencia21.tec.mx/courses/241470/files/78461984/download?download_frd=1) . En los datos escalados, cada píxel representa un área de 10.045 metros x 10.045 metros.

El robot explorador que se desea enviar sólo es capaz de utilizar información de su alrededor para tomar decisiones. Es decir, sólo conocerá la profundidad en su vecindad. Por ello, los ingenieros decidieron que en la simulaciones, si el explorador se encuentra en cierto pixel de la imagen, sólo será capaz de conocer la profundidad en los ocho pixeles vecinos. Además, para evitar daños en el robot, no se permite que el explorador se mueva a pixeles con una diferencia de altura mayor a 0.5 metros de la posición en donde se encuentre.

Para llevar a cabo las simulaciones del descenso, como una primera aproximación, se desea probar algoritmos de búsqueda local tales como búsqueda codiciosa y recocido simulado como mecanismos de toma de decisiones del robot. Para ello, realicen lo siguiente.

### Búsqueda codiciosa

Implementen un algoritmo de búsqueda codiciosa o voraz, con el cual, dado una posición inicial en la imagen del cráter, el explorador busque un camino de descenso para llegar a la parte del cráter más profunda. Una manera de implementar dicho algoritmo sería la siguiente:

- Dado el pixel donde se encuentre el explorador, verifique las profundidades de los ocho pixeles vecinos de dicha posición.
- Escoja el pixel que tenga la menor profundidad posible, pero que la diferencia entre la altura actual y la de la posición vecina no sea mayor a 2.0 metros. Si esta profundidad es mejor que la actual, mueva el explorador a dicha posición.
- Repita lo mismo hasta que el explorador no sea capaz de encontrar una mejor posición.

Prueben su algoritmo en la posición x = 3350 metros y = 5800 metros. ¿Hasta donde es capaz el explorador de llegar? Prueben su algoritmo con otras cinco posiciones cercanas y lejanas al fondo del cráter. ¿Qué tan bueno es el algoritmo para llegar al fondo del cráter es los casos probados?

### Búsqueda con recocido simulado

Utilicen el algoritmo de recocido simulado para encontrar una ruta que lleve al fondo del cráter con el explorador. Cada iteración del algoritmo la pueden implementar de la siguiente manera:

- Dado el pixel donde se encuentre el explorador, seleccione uno de los pixeles vecinos de manera aleatoria, con la condición que la diferencia de alturas entre la posición actual y la del vecino no sea mayor a 0.5 metros.
- Si la nueva posición es mejor que la anterior, mueva el explorador a la posición anterior. De lo contrario, pruebe con la regla de recocido simulado si se acepta dicha nueva posición

Realicen las mismas pruebas llevadas a cabo búsqueda codiciosa. ¿Qué algoritmo logra llegar más profundo en el cráter? ¿Recomendarían a los ingenieros del robot utilizar alguno de estos algoritmos?

Al final, cada integrante del equipo deberá incluir sus contribuciones y conclusiones.

### Dependencias

In [1]:
from simpleai.search import SearchProblem
from simpleai.search import greedy, simulated_annealing
import numpy as np
import pandas as pd

from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Cascadia Code; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")


### Condiciones iniciales

In [2]:
SCALE = 10.0174
crater_map = np.load('crater_map.npy')
NR, NC = crater_map.shape

# Start positions
x0, y0 = 3350, 5800
MAX_DIFF = 0.5
UNTRAVERSABLE = -1
STEP_SIZE = 1


### Funciones auxiliares

In [3]:
def pos_to_index(pos: list[int]) -> list[int]:
    nr = NR - round(pos[0] / SCALE)
    nc = round(pos[1] / SCALE)
    return [nr, nc]


def coord_to_state(coord: list[int]) -> str:
    return f'{coord[0]},{coord[1]}'


def state_to_coord(state: str) -> list[int]:
    return tuple([int(x) for x in state.split(',')])


def report(result):
    print(f'Found path with length {len(result.path())}')
    print(f'Path: {result.path()}')
    print(f'Cost: {result.cost}')


# def tester(search_method, imax, delta=20):

#     print(f'Testing {search_method.__name__}')
#     for i in np.linspace(imax, 0, delta):
#         problem = mars_route_finding(START_STATE, GOAL_STATE, mars_map, i * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)
#         result = search_method(problem, graph_search=True)
#         report(result)
#         print(f'Max difference (factor {i}): {i * MAX_DIFF}')


def distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)


## Implementación

In [82]:
class mars_descent(SearchProblem):

    def __init__(self, initial_state: str, mars_map: np.array, max_height: float, not_traversable: int, step_size: int):

        self.mars_map = mars_map
        self.initial_state = initial_state
        # Seems very convoluted, but this line of code finds the index associated 
        # with the smallest value on the array (not including the untraversable values)
        # self.goal_state = coord_to_state(np.unravel_index(np.argmin(np.sqrt(crater_map * crater_map), axis=None), crater_map.shape))
        self.max_height = max_height
        self.not_traversable = not_traversable
        self.step_size = step_size

        print("Initial state: ", self.initial_state)
        # print("Goal state: ", self.goal_state)

        SearchProblem.__init__(self, initial_state)

    def actions(self, state):
        mars_map = self.mars_map
        actions = []

        pos_directions = ['U', 'D', 'L', 'R', 'UL', 'UR', 'DL', 'DR']
        x0, y0 = state_to_coord(state)

        for action in pos_directions:
            x, y = state_to_coord(self.result(state, action))
            height_difference = abs(mars_map[x0, y0] - mars_map[x, y])

            if (mars_map[x, y] != self.not_traversable):
                if height_difference <= self.max_height:
                    # print(f'[{x}, {y}], action: {action}, height_difference: {height_difference}')
                    actions.append(action)
    
        return actions

    def result(self, state, action):
        x, y = state_to_coord(state)
        step_size = self.step_size

        if action.count('U'):
            x -= step_size
        elif action.count('D'):
            x += step_size
        elif action.count('L'):
            y -= step_size
        elif action.count('R'):
            y += step_size
        elif action.count('UL'):
            x -= step_size
            y -= step_size
        elif action.count('UR'):
            x -= step_size
            y += step_size
        elif action.count('DL'):
            x += step_size
            y -= step_size
        elif action.count('DR'):
            x += step_size
            y += step_size
        else:
            print('No se encontró la acción')
        
        new_state = coord_to_state([x, y])
        # print(action)
        # print(new_state, self.mars_map[state_to_coord(new_state)])
        return new_state

    def is_goal(self, state):
        # check if the current height is smallest possible
        tol = 1
        return self.mars_map[state_to_coord(state)] < tol

    def cost(self, state, action, state2):
        alpha, beta = 1, 0

        x0, y0 = state_to_coord(state)
        xn, yn = state_to_coord(state2)

        distance = np.sqrt((x0 - xn)**2 + (y0 - yn)**2)
        height_difference = abs(self.mars_map[xn, yn] - self.mars_map[x0, y0])

        return height_difference

    def heuristic(self, state):
        # Minimize height ? We already know the position with the lowest height
        x, y = state_to_coord(state)
        # xg, yg = state_to_coord(self.goal_state)
        # distance = np.sqrt((x - xg)**2 + (y - yg)**2)
        # A heuristic that returns the current height?

        return self.mars_map[x, y]

In [83]:
START_STATE = coord_to_state(pos_to_index([x0, y0]))

In [90]:
crater_problem = mars_descent(START_STATE, crater_map, 10 * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)

Initial state:  743,579


In [91]:
res = greedy(crater_problem, graph_search=True)

Al momento de detener la corrida llevaba más de 80 minutos

In [86]:
res.path()

[(None, '743,579'),
 ('D', '744,579'),
 ('D', '745,579'),
 ('L', '745,578'),
 ('D', '746,578'),
 ('L', '746,577'),
 ('L', '746,576'),
 ('U', '745,576'),
 ('L', '745,575'),
 ('U', '744,575'),
 ('U', '743,575'),
 ('U', '742,575'),
 ('U', '741,575'),
 ('U', '740,575'),
 ('U', '739,575'),
 ('U', '738,575'),
 ('R', '738,576'),
 ('R', '738,577'),
 ('U', '737,577'),
 ('U', '736,577'),
 ('R', '736,578'),
 ('R', '736,579'),
 ('R', '736,580'),
 ('R', '736,581'),
 ('R', '736,582'),
 ('U', '735,582'),
 ('R', '735,583'),
 ('U', '734,583'),
 ('U', '733,583'),
 ('U', '732,583'),
 ('U', '731,583'),
 ('R', '731,584'),
 ('R', '731,585'),
 ('U', '730,585'),
 ('U', '729,585'),
 ('R', '729,586'),
 ('U', '728,586'),
 ('R', '728,587'),
 ('U', '727,587'),
 ('U', '726,587'),
 ('U', '725,587'),
 ('U', '724,587'),
 ('U', '723,587'),
 ('U', '722,587'),
 ('U', '721,587'),
 ('U', '720,587'),
 ('U', '719,587'),
 ('L', '719,586'),
 ('U', '718,586'),
 ('U', '717,586'),
 ('L', '717,585'),
 ('U', '716,585'),
 ('U', '715

In [87]:
res.cost

756.6174566406256

In [88]:
res.path()[-1][1]

'507,654'

In [89]:
crater_map[state_to_coord(res.path()[-1][1])]
# Minimal height? I saw ones that were around 100

0.4856029296875022

In [80]:
df = pd.DataFrame(np.sqrt(crater_map * crater_map).flatten())

In [81]:
df.describe()

,0
count,772209.000000
mean,113.165327
std,60.010918
min,0.096675
25%,104.883894
50%,144.555757
75%,152.445222
max,168.222249


In [52]:
df.quantile(q=0.21)

0    24.808622
Name: 0.21, dtype: float64

## Contribuciones

- Juan Pablo Echeagaray González

## Conclusiones

- Juan Pablo Echeagaray González